<a href="https://colab.research.google.com/github/jodfedlet/TCC/blob/main/TCCII/Twitter/preprocessing_tweets_until_dec2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importando/instalando bibliotecas

In [136]:
!pip install unidecode
import pandas as pd
import nltk
import re
import spacy
from nltk.tokenize import word_tokenize
from google.colab import drive
drive.mount("/content/drive")
nltk.download('all-corpora')
nltk.download('punkt')

!python3 -m spacy download pt
!pip install spacy-langdetect

from nltk.corpus import stopwords
from unidecode import unidecode
from spacy_langdetect import LanguageDetector


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /roo

# Carregamento dos tweets até dezembro de 2021

In [234]:
tweets = pd.read_csv('/content/drive/My Drive/ColabNotebooks/tweets_until_dec_2021.csv', lineterminator='\n', index_col=0)

#Quantidade de tweets até dezembro de 2021 antes do pré-processamento

In [235]:
tweets.shape

(174293, 4)

In [236]:
tweets.head()

,datetime,tweet_id,user_id,text
10929,2021-12-30 23:59:11+00:00,1476704451339640832,1907518614,@UOL “Capacetes brancos ajudaram no Haiti”. Pa...
10930,2021-12-30 23:42:29+00:00,1476700246038257672,1306025410661158912,@Jezicacardoso1 @blitz1999_twins @Jacksontoled...
10931,2021-12-30 23:22:01+00:00,1476695096603066369,2742994119,Tive a viagem de uber mais legal. O motorista ...
10932,2021-12-30 22:48:50+00:00,1476686746708492290,912815654201380864,@_C4ssia Se puder ser convertido em kwacha zam...
10933,2021-12-30 22:39:57+00:00,1476684512021172229,66831264,@AragaoMosart Tenente o exército brasileiro fr...


In [238]:
tweets.tail()

,datetime,tweet_id,user_id,text
185217,2018-01-01 11:15:07+00:00,947788269168857088,478396440,"Via #secoursdeshommes, asso #humanitaire #Guip..."
185218,2018-01-01 10:30:07+00:00,947776946787831808,824618261392359425,A internet brasileira é mais lenta que a da Et...
185219,2018-01-01 04:46:44+00:00,947690530498138112,2654726438,https://t.co/1k54jvSxHj\n\nRÉGINE MINHA FADA D...
185220,2018-01-01 01:10:25+00:00,947636095033970688,784508732969512960,@lmjrosez axanei de dedcobir q sei falat haitiano
185221,2018-01-01 01:08:24+00:00,947635584792694785,4039768811,@signofzain anjo tava p traduzir do haitiano e...


In [239]:
tweets['text']

10929     @UOL “Capacetes brancos ajudaram no Haiti”. Pa...
10930     @Jezicacardoso1 @blitz1999_twins @Jacksontoled...
10931     Tive a viagem de uber mais legal. O motorista ...
10932     @_C4ssia Se puder ser convertido em kwacha zam...
10933     @AragaoMosart Tenente o exército brasileiro fr...
                                ...                        
185217    Via #secoursdeshommes, asso #humanitaire #Guip...
185218    A internet brasileira é mais lenta que a da Et...
185219    https://t.co/1k54jvSxHj\n\nRÉGINE MINHA FADA D...
185220    @lmjrosez axanei de dedcobir q sei falat haitiano
185221    @signofzain anjo tava p traduzir do haitiano e...
Name: text, Length: 174293, dtype: object

#Pegar os stop words da língua portuguesa usando NLTK

In [240]:
stop_words = set(stopwords.words('portuguese'))

In [241]:
type(stop_words)


set

In [227]:
stop_words

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houveríamos',
 'houvesse',


#Pré-processamento dos tweets

###Função para limpar os tweets

In [243]:
def pre_processing_tweets(tweet):
  tweet_list = str(tweet).lower().split()
 
  #remover palavras que iniciam com @, #, stopwords ou com tamanho < 2 se forem diferentes de ht.
  def remove_undesirable_words(tweet_list):
      new_list_ = []
      for token in tweet_list:
         if token.startswith('@') or token.startswith('#') or token in stop_words or (len(token) < 3 and token != 'ht'): continue
         new_list_.append(remove_emphasis(token))
      return new_list_

  #remover acentuações
  def remove_emphasis(token):
      return unidecode(token)

  #remover urls
  def remove_url(tweet_without_sw):
      return re.sub(r"http\S+", "", tweet_without_sw)

  #remover pontuações
  def remove_punctuation(tweet):
      return re.sub(r'[^\w\s]','',tweet)

  #remover palavras que contem ou que são números
  def remove_numeric(tweet):
     return ' '.join([token for token in tweet.split() if not token.isdigit() and not any(map(str.isdigit, token))])    

  tweet_list = remove_undesirable_words(tweet_list)
  tweet = remove_url(' '.join(tweet_list))
  tweet = remove_punctuation(tweet)
  tweet = remove_numeric(tweet)
  return ' '.join(remove_undesirable_words(tweet.split()))
 


###Exemplo de tweet antes da aplicação da função de pré-procesamento

In [244]:
string_example = "@gen_heleno VAI BOTAR UM PIJAMA???? This dog 😂😂😂😂 E VAI PRA CASA DESCANSAR no ht vc.  REZA456 E PEÇA A DEUS QUE PERDOE TODAS AS BARBARIDADES QUE VOCÊ COMETEU NO HAITI em 2015. EXISTEM ALMAS LÁ QUE NUNCA IRÃO PERDOAR AS SUAS ATITUDES. VOCÊ E DEUS SABEM DISTO."

In [245]:
string_example

'@gen_heleno VAI BOTAR UM PIJAMA???? This dog 😂😂😂😂 E VAI PRA CASA DESCANSAR no ht vc.  REZA456 E PEÇA A DEUS QUE PERDOE TODAS AS BARBARIDADES QUE VOCÊ COMETEU NO HAITI em 2015. EXISTEM ALMAS LÁ QUE NUNCA IRÃO PERDOAR AS SUAS ATITUDES. VOCÊ E DEUS SABEM DISTO.'

In [ ]:
  # string_ = string_example.split()

  # mot_original = ''
  # for mot in string_:
  #   if not mot.startswith('@') and not mot.startswith('#'):
  #     mot_original += mot.lower()+' '

  # mot_original = nltk.word_tokenize(mot_original, language="portuguese")

In [ ]:
#mot_original

In [246]:
filtered_tweet = pre_processing_tweets(string_example)

###Exemplo de tweet depois da aplicação da função de pré-procesamento

In [247]:
filtered_tweet

'vai botar pijama this dog vai pra casa descansar ht peca deus perdoe todas barbaridades cometeu haiti existem almas nunca irao perdoar atitudes deus sabem disto'

###Aplicação da função do pré-processamento nos tweets

In [248]:
tweets['text'] = tweets['text'].map(lambda tweet: pre_processing_tweets(tweet))

In [249]:
tweets.shape

(174293, 4)

###Remover tweets com menos de 3 palavras

In [252]:
pre_processed_tweets = tweets.loc[tweets['text'].str.split().str.len() > 3]

In [253]:
pre_processed_tweets.shape

(160206, 4)

In [254]:
pre_processed_tweets.head()

,datetime,tweet_id,user_id,text
10929,2021-12-30 23:59:11+00:00,1476704451339640832,1907518614,capacetes brancos ajudaram haiti pare besteira...
10930,2021-12-30 23:42:29+00:00,1476700246038257672,1306025410661158912,anos poder abandonou cinquenta cento populacao...
10931,2021-12-30 23:22:01+00:00,1476695096603066369,2742994119,viagem uber legal motorista haiti professor ve...
10932,2021-12-30 22:48:50+00:00,1476686746708492290,912815654201380864,puder ser convertido kwacha zambiano won norte...
10933,2021-12-30 22:39:57+00:00,1476684512021172229,66831264,tenente exercito brasileiro fracassou haiti


###Remover os tweets duplicados

In [255]:
pre_processed_tweets.drop_duplicates(subset = ["text"], keep='first', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


###Quantidade de tweets até dezembro de 2021 após o pré-processamento

In [186]:
pre_processed_tweets.shape

(140977, 4)

In [257]:
pre_processed_tweets.head()

,datetime,tweet_id,user_id,text
10929,2021-12-30 23:59:11+00:00,1476704451339640832,1907518614,capacetes brancos ajudaram haiti pare besteira...
10930,2021-12-30 23:42:29+00:00,1476700246038257672,1306025410661158912,anos poder abandonou cinquenta cento populacao...
10931,2021-12-30 23:22:01+00:00,1476695096603066369,2742994119,viagem uber legal motorista haiti professor ve...
10932,2021-12-30 22:48:50+00:00,1476686746708492290,912815654201380864,puder ser convertido kwacha zambiano won norte...
10933,2021-12-30 22:39:57+00:00,1476684512021172229,66831264,tenente exercito brasileiro fracassou haiti


In [260]:
pre_processed_tweets.tail()

,datetime,tweet_id,user_id,text
185215,2018-01-01 11:33:11+00:00,947792815668514816,896213196310183936,brasileiros morrem fome brasileiros inocentes ...
185216,2018-01-01 11:27:30+00:00,947791387797393408,182945726,merece homenagem medica sanitarista zilda arns...
185219,2018-01-01 04:46:44+00:00,947690530498138112,2654726438,regine fada haiti escolhi amar
185220,2018-01-01 01:10:25+00:00,947636095033970688,784508732969512960,axanei dedcobir sei falat haitiano
185221,2018-01-01 01:08:24+00:00,947635584792694785,4039768811,anjo tava traduzir haitiano assim portugues ks...


In [262]:
pre_processed_tweets

,datetime,tweet_id,user_id,text
10929,2021-12-30 23:59:11+00:00,1476704451339640832,1907518614,capacetes brancos ajudaram haiti pare besteira...
10930,2021-12-30 23:42:29+00:00,1476700246038257672,1306025410661158912,anos poder abandonou cinquenta cento populacao...
10931,2021-12-30 23:22:01+00:00,1476695096603066369,2742994119,viagem uber legal motorista haiti professor ve...
10932,2021-12-30 22:48:50+00:00,1476686746708492290,912815654201380864,puder ser convertido kwacha zambiano won norte...
10933,2021-12-30 22:39:57+00:00,1476684512021172229,66831264,tenente exercito brasileiro fracassou haiti
...,...,...,...,...
185215,2018-01-01 11:33:11+00:00,947792815668514816,896213196310183936,brasileiros morrem fome brasileiros inocentes ...
185216,2018-01-01 11:27:30+00:00,947791387797393408,182945726,merece homenagem medica sanitarista zilda arns...
185219,2018-01-01 04:46:44+00:00,947690530498138112,2654726438,regine fada haiti escolhi amar
185220,2018-01-01 01:10:25+00:00,947636095033970688,784508732969512960,axanei dedcobir sei falat haitiano


##Testando o idioma da palavra

In [ ]:

pln = spacy.load('pt')
pln

In [ ]:
pln.add_pipe(LanguageDetector(), name='language_detector', last=True)


In [ ]:
texto = 'teste de idioma'
doc = pln(texto)
print(doc._.language)

{'language': 'pt', 'score': 0.9999936219080354}


##Criando o arquivo .csv dos tweets

In [263]:
df = pd.DataFrame(pre_processed_tweets)
df.to_csv('/content/drive/My Drive/ColabNotebooks/pre_processed_tweets_until_dec_2021.csv')